# HW 4: Natural Language Processing

<div class="alert alert-block alert-warning">Each assignment needs to be completed independently. Never ever copy others' work or let someone copy your solution (even with minor modification, e.g. changing variable names). Anti-Plagiarism software will be used to check all submissions. No last minute extension of due date. Be sure to start working on it ASAP! </div>

## Q1: Extract data using regular expression
Suppose you have scraped the text shown below from an online source (https://www.google.com/finance/). 
Define a `extract` function which:
- takes a piece of text (in the format of shown below) as an input
- uses regular expression to transform the text into a DataFrame with columns: 'Ticker','Name','Article','Media','Time','Price',and 'Change' 
- returns the DataFrame

In [3]:
import pandas as pd
import nltk
from sklearn.metrics import pairwise_distances
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import normalize
from nltk.tokenize import RegexpTokenizer
import re
import sklearn
import spacy

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
text = '''QQQ
Invesco QQQ Trust Series 1
Invesco Expands QQQ Innovation Suite to Include Small-Cap ETF
PR Newswire • 4 hours ago
$265.62
1.13%
add_circle_outline
AAPL
Apple Inc
Estimating The Fair Value Of Apple Inc. (NASDAQ:AAPL)
Yahoo Finance • 4 hours ago
$140.41
1.50%
add_circle_outline
TSLA
Tesla Inc
Could This Tesla Stock Unbalanced Iron Condor Return 23%?
Investor's Business Daily • 1 hour ago
$218.30
0.49%
add_circle_outline
AMZN
Amazon.com, Inc.
The Regulators of Facebook, Google and Amazon Also Invest in the Companies' Stocks
Wall Street Journal • 2 days ago
$110.91
1.76%
add_circle_outline'''



In [5]:
def extract(text):
    
    
    df = pd.DataFrame(columns = ["Ticker", "Name", "Article", "Media", "Time", "Price", "Change"])
    
    # split text
    individual_companies = re.split('\nadd_circle_outline', text)
    individual_companies = individual_companies[0:len(individual_companies)-1]

    for company in individual_companies:
        company1 = company.split('\n')
        
        
        while '' in company1:
            company1.remove('')
            
        #print(company1)
        
        company2 = []
        for component in company1:
            if '•' in component:
                article_date = component.split('•')
                company2.append(article_date[0])
                company2.append(article_date[1])

            else:
                company2.append(component)
        print(company2)
        df.loc[len(df)] = company2
            
    result = df
    
    return result

In [6]:
# test your function

extract(text)

['QQQ', 'Invesco QQQ Trust Series 1', 'Invesco Expands QQQ Innovation Suite to Include Small-Cap ETF', 'PR Newswire ', ' 4 hours ago', '$265.62', '1.13%']
['AAPL', 'Apple Inc', 'Estimating The Fair Value Of Apple Inc. (NASDAQ:AAPL)', 'Yahoo Finance ', ' 4 hours ago', '$140.41', '1.50%']
['TSLA', 'Tesla Inc', 'Could This Tesla Stock Unbalanced Iron Condor Return 23%?', "Investor's Business Daily ", ' 1 hour ago', '$218.30', '0.49%']
['AMZN', 'Amazon.com, Inc.', "The Regulators of Facebook, Google and Amazon Also Invest in the Companies' Stocks", 'Wall Street Journal ', ' 2 days ago', '$110.91', '1.76%']


,Ticker,Name,Article,Media,Time,Price,Change
0,QQQ,Invesco QQQ Trust Series 1,Invesco Expands QQQ Innovation Suite to Includ...,PR Newswire,4 hours ago,$265.62,1.13%
1,AAPL,Apple Inc,Estimating The Fair Value Of Apple Inc. (NASDA...,Yahoo Finance,4 hours ago,$140.41,1.50%
2,TSLA,Tesla Inc,Could This Tesla Stock Unbalanced Iron Condor ...,Investor's Business Daily,1 hour ago,$218.30,0.49%
3,AMZN,"Amazon.com, Inc.","The Regulators of Facebook, Google and Amazon ...",Wall Street Journal,2 days ago,$110.91,1.76%


## Q2: Analyze a document

When you have a long document, you would like to 
- Quanitfy how `concrete` a sentence is
- Create a concise summary while preserving it's key information content and overall meaning. Let's implement an `extractive method` based on the concept of TF-IDF. The idea is to identify the key sentences from an article and use them as a summary. 


Carefully follow the following steps to achieve these two targets.

### Q2.1. Preprocess the input document 

Define a function `proprocess(doc, lemmatized = True, remove_stopword = True, lower_case = True, remove_punctuation = True, pos_tag = False)` 
- Four input parameters:
    - `doc`: an input string (e.g. a document)
    - `lemmatized`: an optional boolean parameter to indicate if tokens are lemmatized. The default value is True (i.e. tokens are lemmatized).
    - `remove_stopword`: an optional boolean parameter to remove stop words. The default value is True, i.e., remove stop words. 
    - `remove_punctuation`: optional boolean parameter to remove punctuations. The default values is True, i.e., remove all punctuations.
    - `lower_case`: optional boolean parameter to convert all tokens to lower case. The default option is True, i.e., lowercase all tokens.
    - `pos_tag`: optional boolean parameter to add a POS tag for each token. The default option is False, i.e., no POS tagging.  
    
       
- Split the input `doc` into sentences. Hint, typically, `\n\n+` is used to separate paragraphs. Make sure a sentence does not cross over two paragraphs. You can replace `\n\n+` by a `.`


- Tokenize each sentence into unigram tokens and also process the tokens as follows:
    - If `lemmatized` is True, lemmatize all unigrams. 
    - If `remove_stopword` is set to True, remove all stop words. 
    - If `remove_punctuation` is set to True, remove all punctuations. 
    - If `lower_case` is set to True, convert all tokens to lower case 
    - If `pos_tag` is set to True, find the POS tag for each token and form a tuple for each token, e.g., ('recently', 'ADV'). Either Penn tags or Universal tags are fine. See mapping of these two tagging systems here: https://universaldependencies.org/tagset-conversion/en-penn-uposf.html


- Return the original sentence list (`sents`) and also the tokenized (or tagged) sentence list (`tokenized_sents`). 

   
(Hint: you can use [nltk](https://www.nltk.org/api/nltk.html) and [spacy](https://spacy.io/api/token#attributes) package for this task.)

In [5]:
nlp = spacy.load("en_core_web_sm")

def preprocess(doc, lemmatized=True, pos_tag = False, remove_stopword=True, lower_case = True, remove_punctuation = True):

    # output variables
    sents, tokenized_sents = None, []
    
    # construct a language object
    doc = nlp(doc)
    doc_raw = str(doc)
    doc_raw = re.split('\n\n+', doc_raw)
    doc_raw = ' '.join(doc_raw)
    sents_raw = nltk.sent_tokenize(doc_raw)
    
    # lemmatization
    if lemmatized == True:
        doc = " ".join([token.lemma_ for token in doc])
    else:
        doc = str(doc)
        doc = re.split('\n\n+', doc)
        doc = ' '.join(doc)
        
    if lower_case == True:
        doc = doc.lower()
    
    # tokenizing sentences
    sents = nltk.sent_tokenize(doc)
    
    # removing punctuation
    sents2 = []
    if remove_punctuation == True:
        tokenizer = RegexpTokenizer(r'\w+')
        for sent in sents:
            token = tokenizer.tokenize(sent)
            sent = ' '.join(token)
            sents2.append(sent)
    else:
        sents2 = sents
        

    
    # removing stopword
    tokenized_sents2 = []
    if remove_stopword == True:
        stop_words = nlp.Defaults.stop_words
        for sentence in sents2:
            tokenized_sents.append(nltk.word_tokenize(sentence))
        
        for group in tokenized_sents:
            filtered_sentence = [w for w in group if not w in stop_words]
            tokenized_sents2.append(filtered_sentence)
        
    else:
        for sentence in sents2:
            tokenized_sents2.append(nltk.word_tokenize(sentence))
    
    # position_tagging
    tokenized_sents_final = []
    if pos_tag == True:
        for sent in sents_raw:
            sent = nlp(sent)
            tokenized_sents_final.append([(str(token), token.pos_) for token in sent])
    else:
        tokenized_sents_final = tokenized_sents2
        
    
    # returning the results
    return sents_raw, tokenized_sents_final

In [6]:
# load test document

text = open("power_of_nlp.txt", "r", encoding='utf-8').read()

In [7]:
# test with all default options:

sents, tokenized_sents = preprocess(text)

# print first 3 sentences
for i in range(3):
    print(sents[i], "\n",tokenized_sents[i],"\n\n" )

The Power of Natural Language Processing Until recently, the conventional wisdom was that while AI was better than humans at data-driven decision making tasks, it was still inferior to humans for cognitive and creative ones. 
 ['power', 'natural', 'language', 'processing', 'recently', 'conventional', 'wisdom', 'ai', 'human', 'data', 'drive', 'decision', 'task', 'inferior', 'human', 'cognitive', 'creative'] 


But in the past two years language-based AI has advanced by leaps and bounds, changing common notions of what this technology can do. 
 ['past', 'year', 'language', 'base', 'ai', 'advance', 'leap', 'bound', 'change', 'common', 'notion', 'technology'] 


The most visible advances have been in what’s called “natural language processing” (NLP), the branch of AI focused on how computers can process language like humans do. 
 ['visible', 'advance', 's', 'natural', 'language', 'processing', 'nlp', 'branch', 'ai', 'focus', 'computer', 'process', 'language', 'like', 'human'] 




In [8]:
# process text without remove stopwords, punctuation, lowercase, but with pos tagging

sents, tokenized_sents = preprocess(text, lemmatized = False, pos_tag = True, 
                                    remove_stopword=False, remove_punctuation = False, 
                                    lower_case = False)

for i in range(3):
    print(sents[i], "\n",tokenized_sents[i],"\n\n" )

The Power of Natural Language Processing Until recently, the conventional wisdom was that while AI was better than humans at data-driven decision making tasks, it was still inferior to humans for cognitive and creative ones. 
 [('The', 'DET'), ('Power', 'PROPN'), ('of', 'ADP'), ('Natural', 'PROPN'), ('Language', 'PROPN'), ('Processing', 'NOUN'), ('Until', 'ADP'), ('recently', 'ADV'), (',', 'PUNCT'), ('the', 'DET'), ('conventional', 'ADJ'), ('wisdom', 'NOUN'), ('was', 'AUX'), ('that', 'SCONJ'), ('while', 'SCONJ'), ('AI', 'PROPN'), ('was', 'AUX'), ('better', 'ADJ'), ('than', 'ADP'), ('humans', 'NOUN'), ('at', 'ADP'), ('data', 'NOUN'), ('-', 'PUNCT'), ('driven', 'VERB'), ('decision', 'NOUN'), ('making', 'VERB'), ('tasks', 'NOUN'), (',', 'PUNCT'), ('it', 'PRON'), ('was', 'AUX'), ('still', 'ADV'), ('inferior', 'ADJ'), ('to', 'ADP'), ('humans', 'NOUN'), ('for', 'ADP'), ('cognitive', 'ADJ'), ('and', 'CCONJ'), ('creative', 'ADJ'), ('ones', 'NOUN'), ('.', 'PUNCT')] 


But in the past two years 

### Q2.2. Quantify sentence concreteness


`Concreteness` can increase a message's persuasion. The concreteness can be measured by the use of :
- `article` (e.g., a, an, and the), 
- `adpositions` (e.g., in, at, of, on, etc), and
- `quantifiers`, i.e., adjectives before nouns.


Define a function `compute_concreteness(tagged_sent)` as follows:
- Input argument is `tagged_sent`, a list with (token, pos_tag) tuples as shown above.
- Find the three types of tokens: `articles`, `adposition`, and `quantifiers`.
- Compute `concereness` score as:  `(the sum of the counts of the three types of tokens)/(total non-punctuation tokens)`.
- return the concreteness score, articles, adposition, and quantifiers lists.


Find the most concrete and the least concrete sentences from the article. 


Reference: Peer to Peer Lending: The Relationship Between Language Features, Trustworthiness, and Persuasion Success, https://socialmedialab.sites.stanford.edu/sites/g/files/sbiybj22976/files/media/file/larrimore-jacr-peer-to-peer.pdf

In [9]:
def compute_concreteness(tagged_sent):
    
    concreteness, articles, adpositions,quantifier = None, None, None, []
    
    articles = [tag for tag in tagged_sent if tag[1] == "DET"]
    adpositions = [tag for tag in tagged_sent if tag[1] == "ADP"]
    #quantifier = [tag for tag in tagged_sent if tag[1] == "ADJ"]
    
    for i in range(len(tagged_sent) - 1):
        if tagged_sent[i][1] == "ADJ" and tagged_sent[i+1][1] == "NOUN":
            quantifier.append(tagged_sent[i])
    
    punctuation_tokens = [tag for tag in tagged_sent if tag[1] == "PUNCT"]
    non_punctuation_tokens_count = len(tagged_sent) - len(punctuation_tokens)
    
    concreteness = (len(articles) + len(adpositions) + len(quantifier)) / non_punctuation_tokens_count
    
    return concreteness, articles, adpositions,quantifier
    

In [10]:
# tokenize with pos tag, without change the text much

sents, tokenized_sents = preprocess(text, lemmatized = False, pos_tag = True, 
                                    remove_stopword=False, remove_punctuation = False, 
                                    lower_case = False)

In [11]:
# find concreteness score, articles, adpositions, and quantifiers in a sentence

idx = 1    # sentence id
x = tokenized_sents[idx]
concreteness, articles, adpositions,quantifier = compute_concreteness(x)

# show sentence
sents[idx]
# show result
concreteness, articles, adpositions,quantifier

'But in the past two years language-based AI has advanced by leaps and bounds, changing common notions of what this technology can do.'

(0.25,
 [('the', 'DET'), ('this', 'DET')],
 [('in', 'ADP'), ('by', 'ADP'), ('of', 'ADP')],
 [('common', 'ADJ')])

In [12]:
# Find the most concrete and the least concrete sentences from the article

concrete = []

for token in tokenized_sents:
    score = compute_concreteness(token)[0]
    concrete.append(score)
    
max_id = concrete.index(max(concrete))
min_id = concrete.index(min(concrete))


print (f"The most concerete sentence:  {sents[max_id]}, {concrete[max_id]:.3f}\n")
print (f"The least concerete sentence:  {sents[min_id]}, {concrete[min_id]:.3f}")

The most concerete sentence:  Large foundation models like GPT-3 exhibit abilities to generalize to a large number of tasks without any task-specific training., 0.450

The least concerete sentence:  Remember that while current AI might not be poised to replace managers, managers who understand AI are poised to replace managers who don’t., 0.000


### Q2.3. Generate TF-IDF representations for sentences 

Define a function `compute_tf_idf(sents, use_idf)` as follows: 


- Take the following two inputs:
    - `sents`: tokenized sentences (without pos tagging) returned from Q2.1. These sentences form a corpus for you to calculate `TF-IDF` vectors.
    - `use_idf`: if this option is true, return smoothed normalized `TF_IDF` vectors for all sentences; otherwise, just return normalized `TF` vector for each sentence.
    
    
- Calculate `TF-IDF` vectors as shown in the lecture notes (Hint: you can slightly modify code segment 7.5 in NLP Lecture Notes (II) for this task)

- Return the `TF-IDF` vectors  if `use_idf` is True.  Return the `TF` vectors if `use_idf` is False.

In [13]:
def compute_tf_idf(sents, use_idf = True, min_df = 1):
    
    tf_idf = None
    
    # dictionary of token count
    docs_tokens={idx:nltk.FreqDist(group) \
             for idx,group in enumerate(sents)}
    
    # dataframe from dictionary
    dtm=pd.DataFrame.from_dict(docs_tokens, orient="index")
    dtm=dtm.fillna(0)
    dtm = dtm.sort_index(axis = 0)
    
    # get normalized term frequency (tf) matrix
    tf=dtm.values
    doc_len=tf.sum(axis=1)
    tf=np.divide(tf, doc_len[:,None])
    np.set_printoptions(precision=2)
    
    # get idf
    df=np.where(tf>0,1,0)
    idf=np.log(np.divide(len(tokenized_sents), \
        np.sum(df, axis=0)))+1

    smoothed_idf=np.log(np.divide(len(tokenized_sents)+1, np.sum(df, axis=0)+1))+1
    
    # step 6. get tf-idf

    s = tf*idf
    tf_idf=normalize(tf*idf)

    smoothed_tf_idf=normalize(tf*smoothed_idf)

    
    if use_idf == True:
        return smoothed_tf_idf
    else:
        return tf

In [14]:
# test compute_tf_idf function

sents, tokenized_sents = preprocess(text)
tf_idf = compute_tf_idf(tokenized_sents, use_idf = True)

# show shape of TF-IDF
tf_idf.shape

(78, 485)

### Q2.4. Identify key sentences as summary 

The basic idea is that, in a coherence article, all sentences should center around some key ideas. If we can identify a subset of sentences, denoted as $S_{key}$, which precisely capture the key ideas,  then $S_{key}$ can be used as a summary. Moreover, $S_{key}$ should have high similarity to all the other sentences on average, because all sentences are centered around the key ideas contained in $S_{key}$. Therefore, we can identify whether a sentence belongs to $S_{key}$ by its similarity to all the other sentences.


Define a function `get_summary(tf_idf, sents, topN = 5)`  as follows:

- This function takes three inputs:
    - `tf_idf`: the TF-IDF vectors of all the sentences in a document
    - `sents`: the original sentences corresponding to the TF-IDF vectors
    - `topN`: the top N sentences in the generated summary

- Steps:
    1. Calculate the cosine similarity for every pair of TF-IDF vectors 
    1. For each sentence, calculate its average similarity to all the others 
    1. Select the sentences with the `topN` largest average similarity 
    1. Print the `topN` sentences index
    1. Return these sentences as the summary

In [15]:
def get_summary(tf_idf, sents, topN = 5):
    
    summary = None
    
    cosine_similarity = sklearn.metrics.pairwise.cosine_similarity(tf_idf)
    
    avg_sim = [x.mean() for x in cosine_similarity]
    
    sorted_avg_sim = sorted(avg_sim, reverse = True)
    
    top_N_indices = []
    for val in sorted_avg_sim[0:topN]:
        top_N_indices.append(avg_sim.index(val))
    
    summary = [sents[i] for i in top_N_indices]
    
    
    return summary 

In [16]:
# put everything together and test with different options

sents, tokenized_sents = preprocess(text)
tf_idf = compute_tf_idf(tokenized_sents, use_idf = True)
summary = get_summary(tf_idf, sents, topN = 5)

for sent in summary:
    print(sent,"\n")

Models like GPT-3 are considered to be foundation models — an emerging AI research area — which also work for other types of data such as images and video. 

To begin preparing now, start understanding your text data assets and the variety of cognitive tasks involved in different roles in your organization. 

There is so much text data, and you don’t need advanced models like GPT-3 to extract its value. 

A Language-Based AI Research Assistant In my own work, I’ve been looking at how GPT-3-based tools can assist researchers in the research process. 

Begin incorporating new language-based AI tools for a variety of tasks to better understand their capabilities. 



In [17]:
# Please test summary generated under different configurations


### Q2.5. Analysis 

- Do you think the way to quantify concreteness makes sense? Any other thoughts to measure concreteness or abstractness? Share your ideas in pdf.


- Do you think this method is able to generate a good summary? Any pros or cons have you observed? 


- Do these options `lemmatized, remove_stopword, remove_punctuation, use_idf` matter? 
- Why do you think these options matter or do not matter? 
- If these options matter, what are the best values for these options?


Write your analysis as a pdf file. Be sure to provide some evidence from the output of each step to support your arguments.

### Q2.5. (Bonus 3 points). 


- Can you think a way to improve this extractive summary method? Explain the method you propose for improvement,  implement it, use it to generate a new summary, and demonstrate what is improved in the new summary.


- Or, you can research on some other extractive summary methods and implement one here. Compare it with the one you implemented in Q2.1-Q2.3 and show pros and cons of each method.

## Main block to test all functions

In [18]:
if __name__ == "__main__":  
    
    
    text=text = '''QQQ
Invesco QQQ Trust Series 1
Invesco Expands QQQ Innovation Suite to Include Small-Cap ETF
PR Newswire • 4 hours ago
$265.62
1.13%
add_circle_outline
AAPL
Apple Inc
Estimating The Fair Value Of Apple Inc. (NASDAQ:AAPL)
Yahoo Finance • 4 hours ago
$140.41
1.50%
add_circle_outline
TSLA
Tesla Inc
Could This Tesla Stock Unbalanced Iron Condor Return 23%?
Investor's Business Daily • 1 hour ago
$218.30
0.49%
add_circle_outline
AMZN
Amazon.com, Inc.
The Regulators of Facebook, Google and Amazon Also Invest in the Companies' Stocks
Wall Street Journal • 2 days ago
$110.91
1.76%
add_circle_outline'''
    
    
    print("\n==================\n")
    print("Test Q1")
    print(extract(text))
    
    print("\n==================\n")
    print("Test Q2.1")
    
    text = open("power_of_nlp.txt", "r", encoding='utf-8').read()
    
    sents, tokenized_sents = preprocess(text, lemmatized = False, pos_tag = True, 
                                    remove_stopword=False, remove_punctuation = False, 
                                    lower_case = False)
    
    idx = 1    # sentence id
    x = tokenized_sents[idx]
    concreteness, articles, adpositions,quantifier = compute_concreteness(x)

    # show sentence
    sents[idx]
    # show result
    concreteness, articles, adpositions,quantifier
    
    print("\n==================\n")
    print("Test Q2.2-2.4")
    sents, tokenized_sents = preprocess(text)
    tf_idf = compute_tf_idf(tokenized_sents, use_idf = True)
    summary = get_summary(tf_idf, sents, topN = 5)
    print(summary)




Test Q1
  Ticker                        Name  \
0    QQQ  Invesco QQQ Trust Series 1   
1   AAPL                   Apple Inc   
2   TSLA                   Tesla Inc   
3   AMZN            Amazon.com, Inc.   

                                             Article  \
0  Invesco Expands QQQ Innovation Suite to Includ...   
1  Estimating The Fair Value Of Apple Inc. (NASDA...   
2  Could This Tesla Stock Unbalanced Iron Condor ...   
3  The Regulators of Facebook, Google and Amazon ...   

                        Media          Time    Price Change  
0                PR Newswire    4 hours ago  $265.62  1.13%  
1              Yahoo Finance    4 hours ago  $140.41  1.50%  
2  Investor's Business Daily     1 hour ago  $218.30  0.49%  
3        Wall Street Journal     2 days ago  $110.91  1.76%  


Test Q2.1


'But in the past two years language-based AI has advanced by leaps and bounds, changing common notions of what this technology can do.'

(0.25,
 [('the', 'DET'), ('this', 'DET')],
 [('in', 'ADP'), ('by', 'ADP'), ('of', 'ADP')],
 [('common', 'ADJ')])



Test Q2.2-2.4
['Models like GPT-3 are considered to be foundation models — an emerging AI research area — which also work for other types of data such as images and video.', 'To begin preparing now, start understanding your text data assets and the variety of cognitive tasks involved in different roles in your organization.', 'There is so much text data, and you don’t need advanced models like GPT-3 to extract its value.', 'A Language-Based AI Research Assistant In my own work, I’ve been looking at how GPT-3-based tools can assist researchers in the research process.', 'Begin incorporating new language-based AI tools for a variety of tasks to better understand their capabilities.']
